In [1]:
#生成数据集
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

可以调用框架中现有的API来读取数据。 

我们将features和labels作为API的参数传递，并通过数据迭代器指定batch_size。

 此外，布尔值is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [2]:
#读取数据集
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [3]:
next(iter(data_iter))

[tensor([[ 0.3149, -1.2476],
         [-1.1379, -1.0973],
         [ 1.4925, -0.9296],
         [ 1.3957,  0.4555],
         [-0.7532, -0.9325],
         [-0.6191,  0.3177],
         [ 0.9106, -1.7613],
         [-0.4293, -1.2681],
         [-0.4543, -0.1576],
         [ 2.3890, -0.2164]]),
 tensor([[ 9.0801],
         [ 5.6518],
         [10.3366],
         [ 5.4398],
         [ 5.8702],
         [ 1.8719],
         [11.9979],
         [ 7.6670],
         [ 3.8271],
         [ 9.7304]])]

# 定义模型
对于标准深度学习模型，我们可以使用框架的预定义好的层。

这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。

1. 我们首先定义一个模型变量net，它是一个Sequential类的实例。 
2. Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。 
3. 在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。 
4. 但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。

 这一单层被称为全连接层（fully-connected layer）， 因为它的每一个输入都通过矩阵-向量乘法得到它的每个输出。

In [5]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

在使用net之前，我们需要初始化模型参数。 如在线性回归模型中的权重和偏置。


深度学习框架通常有预定义的方法来初始化参数。

 在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零。

In [6]:
# 正如我们在构造nn.Linear时指定输入和输出尺寸一样，
# 现在我们能直接访问参数以设定它们的初始值
#  我们通过net[0]选择网络中的第一个图层，
# 然后使用weight.data和bias.data方法访问参数。 
# 我们还可以使用替换方法normal_和fill_来重写参数值。


net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [7]:
#定义损失函数，计算均方误差使用的是MSELoss类
loss = nn.MSELoss()


In [8]:
#定义优化算法
#小批量随机梯度下降算法是一种优化神经网络的标准工具，
# PyTorch在optim模块中实现了该算法的许多变种。
# 当我们实例化一个SGD实例时，我们要指定优化的参数 
# （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。
# 小批量随机梯度下降只需要设置lr值，这里设置为0.03
trainer = torch.optim.SGD(net.parameters(), lr=0.03)


In [ ]:
#训练模型
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000188
epoch 2, loss 0.000102
epoch 3, loss 0.000103


In [10]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-8.0824e-05,  9.0170e-04])
b的估计误差： tensor([9.0599e-05])
